# Compare EC profit redistribution

This notebook aims at comparing the profit redistribution for an Energy Community
using the package EnergyCommunity.jl and Games.jl

## Configuration

### Paths

In [1]:
input_file = joinpath(@__DIR__, "../data/energy_community_model.yml")  # Input file

output_file_isolated = joinpath(@__DIR__, "../results/output_file_NC.xlsx")  # Output file - model users alone
output_plot_isolated = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_NC.png")  # Output png file of plot - model users alone

output_file_combined = joinpath(@__DIR__, "../results/output_file_EC.xlsx")  # Output file - model Energy community
output_plot_combined = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_EC.pdf")  # Output png file of plot - model energy community

output_plot_sankey_agg = joinpath(@__DIR__, "../results/Img/sankey_EC.png")  # Output plot of the sankey plot related to the aggregator case
output_plot_sankey_noagg = joinpath(@__DIR__, "../results/Img/sankey_NC.png")  # Output plot of the sankey plot related to the no aggregator case

enum_mode_file = "enum_mode_datasest.jld2"  # file used to store the enumerative results
total_results_file = "total_results_file_nocrossover.jld2"  # file to store all major results
latex_output = "latex_output_nocrossover.txt"

overwrite_files = true  # when true, output files are overwritten

true

### Import libraries

In [2]:
using Revise
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings

### Solver configurations

In [3]:
# General optimizer
OPTIMIZER = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0, "Threads"=>10)

# Optimizer for row-generation techniques, used in the IterMode of Games.jl
OPTIMIZER_ROW_GENERATION = optimizer_with_attributes(Gurobi.Optimizer,
    "OutputFlag"=>1,
    "LogToConsole"=>0,
    "MIPGap"=>0.1,
    # "MIPFocus"=>1,
    "TimeLimit"=>1000,
    "LogFile"=>"gurobi_nocrossover.log",
    "Threads"=>10,
    # "NoRelHeurTime"=>10,
    "PoolSolutions"=>50,
    "PoolSearchMode"=>1,
    "Crossover"=>0,  # disable crossover
)

MathOptInterface.OptimizerWithAttributes(Gurobi.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("OutputFlag") => 1, MathOptInterface.RawOptimizerAttribute("LogToConsole") => 0, MathOptInterface.RawOptimizerAttribute("MIPGap") => 0.1, MathOptInterface.RawOptimizerAttribute("TimeLimit") => 1000, MathOptInterface.RawOptimizerAttribute("LogFile") => "gurobi_nocrossover.log", MathOptInterface.RawOptimizerAttribute("Threads") => 10, MathOptInterface.RawOptimizerAttribute("PoolSolutions") => 50, MathOptInterface.RawOptimizerAttribute("PoolSearchMode") => 1, MathOptInterface.RawOptimizerAttribute("Crossover") => 0])

### Energy Community options

In [4]:
NO_AGG_GROUP = GroupANC();  # type of aggregation when the Aggregator does not belong to the coalition.
                            # options: GroupANC() or GroupNC()
BASE_GROUP = GroupNC();     # base type of aggregation (it shall be GroupNC)

## General

In [5]:
ENV["COLUMNS"] = 1000  # to print more columns in the output

1000

## Optimize the Energy Community in the COperative configuration

In [6]:
# Read data from excel file
ECModel = ModelEC(input_file, EnergyCommunity.GroupCO(), OPTIMIZER)

# Reset the user set to use all stored users (10)
reset_user_set!(ECModel)
# set_user_set!(ECModel, ["user$id" for id=1:8])

# Build the model
build_model!(ECModel)

# Optimize the model
optimize!(ECModel)

Academic license - for non-commercial use only - expires 2023-02-04


An Energy Community Model
Energy Community problem for a Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]


Solved model


## Calculate the reward distribution functions by row-generation techniques (IterMode in Games.jl)

### Define the row-generation decomposition mode by IterMode

In [7]:
iter_mode = IterMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP, optimizer=OPTIMIZER_ROW_GENERATION)

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04


Model not optimized
Solved model

IterMode(Any["EC", "user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"], EnergyCommunity.var"#utility_callback_by_subgroup#664"{EnergyCommunity.var"#objective_callback_by_subgroup#585"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#613"{ModelEC}}, EnergyCommunity.var"#objective_callback_by_subgroup#490"{JuMP.Containers.DenseAxisArray{Float64, 1, Tuple{Vector{Any}}, Tuple{JuMP.Containers._AxisLookup{Dict{Any, Int64}}}}}}(EnergyCommunity.var"#objective_callback_by_subgroup#585"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#613"{ModelEC}}(An Energy Community Model
Energy Community problem for a Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]
, EnergyCommunity.var"#objective_callback_by_subgroup#613"{ModelEC}(An Energy Community Model
Energy Community problem for a Aggregating-Non-Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "u


Model not optimized


Define coalitions to precompute when performing the iterative procedure

In [8]:
# include all coalitions having no more than preload_max_size users
preload_max_size = 3

preload_coalitions = Iterators.flatten([combinations([EC_CODE; ECModel.user_set], k) for k = 1:preload_max_size])

Base.Iterators.Flatten{Vector{Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{Any}}}}}(Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{Any}}}[Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{Any}}}(Combinatorics.var"#reorder#10"{Vector{Any}}(Any["EC", "user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]), Combinatorics.Combinations(11, 1)), Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{Any}}}(Combinatorics.var"#reorder#10"{Vector{Any}}(Any["EC", "user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]), Combinatorics.Combinations(11, 2)), Base.Generator{Combinatorics.Combinations, Combinatorics.var"#reorder#10"{Vector{Any}}}(Combinatorics.var"#reorder#10"{Vector{Any}}(Any["EC", "user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]), Combinatorics.C

### Calculate reward distribution using row-generation technique

Variance Least Core using IterMode

In [9]:
tick()
varleastcore_dist_iter, min_surplus_varleastcore_iter, history_varleastcore_iter, model_dist_varleastcore_iter = var_least_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
)
time_elapsed_varleastcore_iter=tok()
println("Variance Least Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varleastcore_iter/60)")

PHASE 1: Start first least core analysis


┌ Info:  started timer at: 2022-06-12T00:51:37.152
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54



Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only 

Iteration       Upper bound     Current value   Tol. [%]        Benefit distribution


1                1.28e+04       -2.62e+04        148.82         [35049.87589167791, 12788.228275191717, 30793.40952735208, 12788.228275191717, 14423.994622248321, 12788.228275191717, 12788.228275191717, 38501.32148124849, 12788.228275191717, 13030.481510193786, 31409.82241316326]


...


20               2.28e+03        3.79e+02        83.35          [44406.3810589058, 8544.495431180441, 44406.38105890553, 8544.495431180316, 6117.474821140171, 4167.646092585113, 4432.572952050339, 49920.08411853194, 7987.946491287301, 4216.188307170284, 44406.381058905485]


21               2.28e+03        2.25e+02        90.10          [44406.38105890556, 10442.56056661437, 44406.38105890548, 6646.430295746423, 6117.474821140153, 4167.646092585142, 4432.57295205035, 49920.08411853188, 7987.946491287297, 4216.188307170336, 44406.38105890549]


22               2.28e+03        4.80e+02        78.91          [45090.30455630436, 8390.790074417466, 45090.30455630448, 6646.430295746427, 6117.474821140163, 4167.64609258517, 4432.572952050341, 49920.084118531784, 7987.946491287339, 4216.188307170345, 45090.304556304465]


23               2.28e+03        9.89e+02        56.57          [44491.32521693278, 10187.728092532154, 44491.325216932906, 6646.430295746477, 6117.474821140144, 4167.646092585139, 4432.572952050359, 51717.02213664639, 6191.008473172704, 4216.188307170423, 44491.325216932906]


24               2.28e+03        2.28e+03        0.00           [44920.72499461557, 8899.528759484692, 44920.72499461536, 6646.43029574639, 6117.474821140144, 4167.646092585129, 4432.572952050359, 51717.022136646556, 6191.008473172645, 4216.188307170341, 44920.72499461539]
PHASE 2: Completed second phase

Variance Least Core - IterMode calculated with elapsed time [min]: 129.16455776666666


Variance Core method using IterMode

In [10]:
tick()
varcore_dist_iter, min_surplus_varcore_iter, history_varcore_iter, model_dist_varcore_iter = var_in_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
)
time_elapsed_varcore_iter=tok()
println("Variance Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varcore_iter/60)")

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only -

┌ Info:  started timer at: 2022-06-12T03:00:47.785
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54



1               -5.70e+04        0.00e+00        100.00         [25932.341398662502, 18669.127828231864, 18669.127828231845, 21686.514575601326, 21923.482199636488, 19660.600279888426, 18669.127828231845, 25932.341398662546, 18669.127828231853, 18669.127828231845, 18669.12782823185]


2               -3.95e+04        0.00e+00        100.00         [30634.603791526843, 6438.263822483359, 30634.603791526653, 30634.60379152664, 10155.29880923852, 30634.60379152663, 6438.263822483353, 34351.63877828181, 7932.930880743313, 8660.631750978535, 30634.603791526668]


3               -1.03e+04        0.00e+00        100.00         [39836.559259666225, 9906.34727148172, 39836.55925966622, 14089.210348625122, 9906.347271481896, 14089.210348625122, 6.286739800533606e-11, 39836.559259666225, 9906.347271481725, 9906.347271481716, 39836.559259666225]

...


11              -2.96e+02        0.00e+00        100.00         [43039.080596154985, 10180.842854046563, 43039.08059615516, 8923.55977722928, 8040.17682994294, 6444.775574064753, 6385.192794592276, 43039.08059615515, 8525.858818695022, 6493.317788650385, 43039.08059615518]


12              -3.53e+02        0.00e+00        100.00         [43039.08059616135, 9885.263507990143, 43039.080596156295, 8923.559777228711, 8335.756175998386, 6444.7755740617185, 6089.61344853625, 43039.08059615632, 8821.43816475108, 6493.317788649781, 43039.08059615632]


13              -2.92e+02        0.00e+00        100.00         [43039.08059615517, 10238.563718085752, 43039.08059615462, 8923.559777229864, 8335.756175998371, 6444.775574065342, 6089.613448537493, 43039.080596155014, 8468.13795465469, 6493.317788650994, 43039.080596155145]


14              -1.52e+02        0.00e+00        100.00         [43112.01786247127, 9946.814652824936, 43112.01786246923, 8923.559777229866, 8044.007110737563, 6444.775574065356, 6381.362513798312, 43112.017862470144, 8468.137954654689, 6493.31778865099, 43112.01786247009]


15              -5.61e-09        0.00e+00        0.56           [43112.01786246746, 10068.732399251614, 43112.017862470646, 8923.559777229842, 8073.940907944625, 6444.775574065349, 6351.4287165911965, 43112.01786247313, 8346.220208228055, 6493.317788650977, 43112.01786246955]
Variance Core - IterMode calculated with elapsed time [min]: 94.973606795


Store results as a DataFrame

In [11]:
# vector of the users
user_set_agg = [EC_CODE; get_user_set(ECModel)]

"Auxiliary function to order the output of reward distributions and return them as vectors"
vectorize_rewards(reward_dist, users_list=user_set_agg) = [reward_dist[u] for u in users_list]

# dataframe of reward distributions for the enumerative mode
df_reward_iter = DataFrame(
    user_set=user_set_agg,
    varcore_iter=vectorize_rewards(varcore_dist_iter),
    varleastcore_iter=vectorize_rewards(varleastcore_dist_iter),
)

# dataframe of the time requirements
dict_time_iter = Dict(
    "IterMode"=>0.0,
    "varcore_iter"=>time_elapsed_varcore_iter,
    "varleastcore_iter"=>time_elapsed_varleastcore_iter,
)

df_reward_iter

,user_set,varcore_iter,varleastcore_iter
,Any,Float64,Float64
1,EC,43112.0,44920.7
2,user4,10068.7,8899.53
3,user7,43112.0,44920.7
4,user8,8923.56,6646.43
5,user9,8073.94,6117.47
6,user2,6444.78,4167.65
7,user1,6351.43,4432.57
8,user5,43112.0,51717.0
9,user3,8346.22,6191.01


## Group all results and save them

Merge results

In [12]:
df_reward = df_reward_iter # innerjoin(df_reward_enum, df_reward_iter, on=:user_set)
df_reward

,user_set,varcore_iter,varleastcore_iter
,Any,Float64,Float64
1,EC,43112.0,44920.7
2,user4,10068.7,8899.53
3,user7,43112.0,44920.7
4,user8,8923.56,6646.43
5,user9,8073.94,6117.47
6,user2,6444.78,4167.65
7,user1,6351.43,4432.57
8,user5,43112.0,51717.0
9,user3,8346.22,6191.01


In [13]:
dict_time = dict_time_iter # merge(dict_time_enum, dict_time_iter)
dict_time

Dict{String, Float64} with 3 entries:
  "varcore_iter"      => 5698.42
  "IterMode"          => 0.0
  "varleastcore_iter" => 7749.87

Save results

In [14]:
# print to latex the reward table
set_default(fmt = "%.2f", convert_unicode = false)

sorted_user_set = [EC_CODE; ["user$u" for u in 1:length(get_user_set(ECModel))]]

df_reward_mod = copy(df_reward)
sort!(df_reward_mod, [order(:user_set, by=x->findfirst(x .== sorted_user_set))])
df_reward_mod[!, 2:end] = df_reward[!, 2:end] ./ 1000  # change € unit to k€
tex_df_reward = latexify(df_reward_mod; env=:table, latex=false)

L"\begin{tabular}{ccc}
user_set & varcore_iter & varleastcore_iter\\
EC & 43.11 & 44.92\\
user1 & 10.07 & 8.90\\
user2 & 43.11 & 44.92\\
user3 & 8.92 & 6.65\\
user4 & 8.07 & 6.12\\
user5 & 6.44 & 4.17\\
user6 & 6.35 & 4.43\\
user7 & 43.11 & 51.72\\
user8 & 8.35 & 6.19\\
user9 & 6.49 & 4.22\\
user10 & 43.11 & 44.92\\
\end{tabular}
"

In [15]:
df_time = DataFrame(dict_time)[!, names(df_reward)[2:end]]./3600
df_time[!, "title"] = [L"Time [h]"]
df_time = df_time[!, ["title"; names(df_reward)[2:end]]]

,title,varcore_iter,varleastcore_iter
,LaTeXStr…,Float64,Float64
1,$Time [h]$,1.58289,2.15274


In [16]:
df_iterations = DataFrame(
    title=L"Iterations",
    varcore_iter=history_varcore_iter[end][1],
    varleastcore_iter=history_varleastcore_iter[end][1],
)

,title,varcore_iter,varleastcore_iter
,LaTeXStr…,Int64,Int64
1,$Iterations$,14,23


In [17]:
df_computational_time = vcat(df_time, df_iterations)

# print to latex the equivalent dataframe for time
tex_df_computational_time = latexify(df_computational_time; env=:table)

L"\begin{tabular}{ccc}
$title$ & $varcore_{iter}$ & $varleastcore_{iter}$\\
$Time [h]$ & $1.58$ & $2.15$\\
$Iterations$ & $14.00$ & $23.00$\\
\end{tabular}
"

In [18]:
# save latex code
open(latex_output,"w") do io
    println(io, "Reward distribution table\n\n\n")
    print(io, tex_df_reward)
    println(io, "Computational time table\n\n\n")
    print(io, tex_df_computational_time)
end

In [19]:
if !isfile(total_results_file) || overwrite_files
    jldsave(total_results_file; df_reward, dict_time, df_computational_time)
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\Davide\.julia\packages\JLD2\k9Gt0\src\JLD2.jl:233
